Team Members:
- Emily Liang 79453973
- 
- 
-

In [ ]:
# Install required packages
%pip install gradio unstructured sentence-transformers
%pip install google.generativeai     # for using local IDE

In [ ]:
import sys
# print(sys.executable)
!{sys.executable} -m pip install google-generativeai

In [1]:
# Import necessary libraries
import os
import time
from PIL import Image
import google.generativeai as genai

In [ ]:
from google.generativeai import types
import os

# For local environment, directly put your API key here
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)
client = genai.GenerativeModel(model_name="gemini-2.5-flash-lite")